# <center>Rasio Kasus Baru Terhadap Spesimen Covid-19 di Indonesia<center><hr>

In [25]:

import pandas as pd
pd.options.mode.chained_assignment = None
dailyCaseIndo_df = pd.read_excel('dailyCaseIndo.xlsx')

In [26]:
kasusHarian_df = dailyCaseIndo_df[["iso_code", "continent","location", "date", "new_cases_smoothed", "new_tests_smoothed"]]
#Remove Na value
kasusHarian_df.dropna(subset = ["date", "new_cases_smoothed", "new_tests_smoothed"],inplace=True)

In [27]:
ratio_case_per_test = kasusHarian_df.new_cases_smoothed / kasusHarian_df.new_tests_smoothed
kasusHarian_df['ratio_case_per_test'] = ratio_case_per_test


In [28]:
#Create linear model
i = -276
x = []
linearModel_df = kasusHarian_df

for row in linearModel_df['date']:
    x.append(i)
    i += 1    
linearModel_df['x'] = x

xy = linearModel_df.x * linearModel_df.ratio_case_per_test
linearModel_df['xy'] = xy

x2 = linearModel_df.x ** 2
linearModel_df['x2'] = x2
n = len(linearModel_df.index) 
a = linearModel_df.ratio_case_per_test.sum()/n
b = linearModel_df.xy.sum()/linearModel_df.x2.sum()
linearModel_df['y_model'] = a + (b * linearModel_df.x)


In [29]:
#Create Quadratic Model
quadModel_df = linearModel_df
n = len(linearModel_df.index) 
quadModel_df['x4'] = quadModel_df.x2 ** 2
quadModel_df['x2y'] = quadModel_df.x2 * quadModel_df.ratio_case_per_test

a = ((quadModel_df.ratio_case_per_test.sum() * quadModel_df.x4.sum()) - (quadModel_df.x2y.sum() * quadModel_df.x2.sum() ))/((n * quadModel_df.x4.sum())- (quadModel_df.x2.sum() ** 2))
b = quadModel_df.xy.sum()/quadModel_df.x2.sum()
c = ( (n * quadModel_df.x2y.sum())- (quadModel_df.x2.sum() * quadModel_df.ratio_case_per_test.sum()) )/( (n * quadModel_df.x4.sum()) - (quadModel_df.x2.sum()) ** 2 )

quadModel_df['y_model2'] = a + (b*quadModel_df.x) + (c * (quadModel_df.x) **2 )


In [30]:
#Create plot
import plotly.graph_objects as go

df = quadModel_df
fig = go.Figure()

#Actual plot
fig.add_trace(go.Scatter(x=df.date, y=df.ratio_case_per_test, name='Actual plot',
                         line=dict(color='#488f31', width=2)))
#Linear model
fig.add_trace(go.Scatter(x=df.date, y=df.y_model, name='Linear model',
                         line=dict(color='#f4bd6a', width=3, dash='dashdot')))
#Quad model
fig.add_trace(go.Scatter(x=df.date, y=df.y_model2, name='Quadratic model',
                         line=dict(color='#de425b', width=3, dash='dash')))


fig.update_layout(title='Grafik Rasio Kasus Baru Terhadap Spesimen Covid-19 di Indonesia',
                   xaxis_title='Tanggal',
                   yaxis_title='Rasio (kasus baru/spesimen)',
                 )
fig.show()

In [31]:
fig = go.Figure()
df = quadModel_df
#Actual plot
fig.add_trace(go.Scatter(x=df.date, y=df.new_cases_smoothed, name='Actual plot',
                         line=dict(color='#f4bd6a', width=2)))
fig.update_layout(title='Grafik Kasus Baru Covid-19 di Indonesia',
                   xaxis_title='Tanggal',
                   yaxis_title='Kasus baru',
                 )
fig.show()

In [32]:
fig = go.Figure()
df = quadModel_df
#Actual plot
fig.add_trace(go.Scatter(x=df.date, y=df.new_tests_smoothed, name='Actual plot',
                         line=dict(color='#de425b', width=2)))
fig.update_layout(title='Grafik Spesimen Covid-19 di Indonesia',
                   xaxis_title='Tanggal',
                   yaxis_title='Spesimen baru',
                 )
fig.show()

In [33]:

from IPython.core.display import HTML

x =[]
i=1
df = kasusHarian_df[["iso_code", "continent","location", "date", "new_cases_smoothed", "new_tests_smoothed"]]
for row in df['date']:
    x.append(i)
    i += 1
df['index'] = x
df.set_index('index').rename_axis(None)




,iso_code,continent,location,date,new_cases_smoothed,new_tests_smoothed
1,IDN,Asia,Indonesia,2020-03-23,63.571,179.0
2,IDN,Asia,Indonesia,2020-03-24,73.429,230.0
3,IDN,Asia,Indonesia,2020-03-25,80.429,279.0
4,IDN,Asia,Indonesia,2020-03-26,83.143,328.0
5,IDN,Asia,Indonesia,2020-03-27,96.714,370.0
...,...,...,...,...,...,...
549,IDN,Asia,Indonesia,2021-09-22,2930.571,169347.0
550,IDN,Asia,Indonesia,2021-09-23,2892.857,170461.0
551,IDN,Asia,Indonesia,2021-09-24,2710.286,170620.0
552,IDN,Asia,Indonesia,2021-09-25,2532.000,165501.0


In [34]:
maxCase = df.new_cases_smoothed.max()
minCase = df.new_cases_smoothed.min()
maxTest = df.new_tests_smoothed.max()
minTest = df.new_tests_smoothed.min()


HTML("""<table style='font-size:1.5em; border:1px solid black'>
  <tr>
    <td style='text-align:left'>Kasus paling sedikit</td>
    <td>:</td>
    <td>"""+str(minCase)+"""</td>
  </tr>
  <tr>
    <td style='text-align:left'>Kasus paling banyak</td>
    <td>:</td>
    <td>"""+str(maxCase)+"""</td>
  </tr>
  <tr>
  <td style='text-align:left'>Spesimen paling sedikit</td>
    <td>:</td>
    <td>"""+str(minTest)+"""</td>
  </tr>
  <tr>
  <td style='text-align:left'>Spesimen paling banyak</td>
    <td>:</td>
    <td>"""+str(maxTest)+"""</td>
  </tr>
</table>""")


Kasus paling sedikit,:,63.571
Kasus paling banyak,:,50039.0
Spesimen paling sedikit,:,179.0
Spesimen paling banyak,:,170620.0


### Sumber data
* <a>https://ourworldindata.org/</a>

### Penulis
* Fatkhul Mukhlish Al-haq
* Viona Febriana
